# 1D Wave Equation — Numerical Solution & Visualization

We solve the one-dimensional wave equation

$$u_{tt} = c^2 u_{xx}, \quad x \in [0,L]$$

with boundary conditions:
- **Fixed ends**: $u(0,t)=u(L,t)=0$
- **or Periodic**

A second-order finite difference scheme is used:

$$u_i^{n+1} = 2u_i^n - u_i^{n-1} + \left(\frac{c\Delta t}{\Delta x}\right)^2
\left(u_{i+1}^n - 2u_i^n + u_{i-1}^n\right)$$

Stability (CFL condition):

$$\frac{c\,\Delta t}{\Delta x} \le 1$$


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import animation
from IPython.display import HTML, display
from ipywidgets import interact, FloatSlider, IntSlider, Dropdown


In [ ]:
def solve_wave_1d(
    L=1.0,
    c=1.0,
    nx=200,
    dt=0.002,
    t_end=2.0,
    init="pluck",
    bc="fixed",
    damping=0.0,
):
    """Solve u_tt = c^2 u_xx on [0,L] using explicit 2nd-order finite differences.

    Args:
      L: domain length
      c: wave speed
      nx: number of spatial grid points
      dt: time step
      t_end: final simulation time
      init: 'gaussian', 'pluck', or 'sine'
      bc: 'fixed' or 'periodic'
      damping: optional damping coefficient (0 = none)

    Returns:
      x: spatial grid
      times: time grid
      U: solution snapshots, shape (nt, nx)
      r: CFL number c*dt/dx
    """

    x = np.linspace(0, L, nx)
    dx = x[1] - x[0]
    r = c * dt / dx

    if r > 1.0:
        raise ValueError(f"CFL violated: r={r:.3f} > 1. Reduce dt or increase dx.")

    nt = int(t_end / dt) + 1
    times = np.linspace(0, dt*(nt-1), nt)

    # Initial displacement u(x,0)
    if init == "gaussian":
        u0 = np.exp(-((x - 0.35*L)/(0.07*L))**2)
    elif init == "pluck":
        u0 = np.where(x < 0.35*L, x/(0.35*L), (L-x)/(L-0.35*L))
        u0 = np.clip(u0, 0, 1)
    elif init == "sine":
        u0 = np.sin(np.pi*x/L)
    else:
        raise ValueError("Invalid init. Use 'gaussian', 'pluck', or 'sine'.")

    # Initial velocity u_t(x,0)
    v0 = np.zeros_like(x)

    U = np.zeros((nt, nx), dtype=float)

    def laplacian(u):
        if bc == "periodic":
            return (np.roll(u,-1) - 2*u + np.roll(u,1)) / dx**2
        elif bc == "fixed":
            uxx = np.zeros_like(u)
            uxx[1:-1] = (u[2:] - 2*u[1:-1] + u[:-2]) / dx**2
            return uxx
        else:
            raise ValueError("Invalid bc. Use 'fixed' or 'periodic'.")

    def apply_bc(u):
        if bc == "fixed":
            u[0] = 0.0
            u[-1] = 0.0

    # Initialize u^0
    u_nm1 = u0.copy()  # will hold u^{n-1}
    u_n = u0.copy()    # will hold u^{n}
    apply_bc(u_n)

    # Initialize u^1 using Taylor expansion
    u_1 = u0 + dt*v0 + 0.5*(c*dt)**2 * laplacian(u0)
    apply_bc(u_1)

    U[0] = u_n
    if nt > 1:
        U[1] = u_1

    gamma = float(damping)

    # Time stepping
    u_nm1, u_n = u_n, u_1

    for n in range(1, nt-1):
        uxx = laplacian(u_n)
        u_np1 = 2*u_n - u_nm1 + (c*dt)**2 * uxx

        # Simple damping: u_tt + 2*gamma*u_t = c^2 u_xx
        if gamma > 0:
            u_np1 -= 2*gamma*dt*(u_n - u_nm1)

        apply_bc(u_np1)
        U[n+1] = u_np1
        u_nm1, u_n = u_n, u_np1

    return x, times, U, r


In [ ]:
# Run one simulation (edit these defaults as you like)
x, times, U, r = solve_wave_1d(init="pluck", bc="fixed")
print(f"CFL number r = {r:.3f} (<= 1 for stability)")

# Plot a few snapshots
plt.figure(figsize=(10,4))
for k in [0, len(times)//4, len(times)//2, -1]:
    plt.plot(x, U[k], label=f"t={times[k]:.2f}")
plt.xlabel("x")
plt.ylabel("u(x,t)")
plt.title("Wave Equation Snapshots")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
def animate_solution(x, times, U, interval=20):
    fig, ax = plt.subplots(figsize=(10,4))
    line, = ax.plot([], [], lw=2)
    ax.set_xlim(x[0], x[-1])
    ax.set_ylim(U.min()*1.1, U.max()*1.1)
    ax.set_xlabel("x")
    ax.set_ylabel("u(x,t)")
    ax.grid(True)
    title = ax.set_title("")

    def update(frame):
        line.set_data(x, U[frame])
        title.set_text(f"1D Wave Equation   t = {times[frame]:.3f}")
        return line, title

    ani = animation.FuncAnimation(
        fig, update, frames=len(times),
        interval=interval, blit=True
    )

    plt.close(fig)
    return HTML(ani.to_jshtml())

display(animate_solution(x, times, U, interval=20))


In [ ]:
@interact(
    init=Dropdown(options=["gaussian","pluck","sine"], value="pluck", description="init"),
    bc=Dropdown(options=["fixed","periodic"], value="fixed", description="bc"),
    c=FloatSlider(min=0.2, max=3.0, step=0.1, value=1.0, description="c"),
    nx=IntSlider(min=80, max=500, step=20, value=200, description="nx"),
    dt=FloatSlider(min=0.0005, max=0.01, step=0.0005, value=0.002, description="dt"),
    t_end=FloatSlider(min=0.5, max=5.0, step=0.5, value=2.0, description="t_end"),
    damping=FloatSlider(min=0.0, max=1.0, step=0.05, value=0.0, description="damping"),
)
def run_demo(init, bc, c, nx, dt, t_end, damping):
    try:
        x, times, U, r = solve_wave_1d(
            L=1.0, c=c, nx=nx, dt=dt,
            t_end=t_end, init=init, bc=bc, damping=damping
        )
        print(f"CFL r = {r:.3f} (<= 1 stable)")
        display(animate_solution(x, times, U, interval=20))
    except Exception as e:
        print("Error:", e)
        print("Tip: If CFL is violated, decrease dt, decrease c, or reduce nx.")
